In [1]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
time.sleep(0.2)

# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1.

### Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

In [179]:
URL_habr = 'http://habr.com/ru/all/'
KEYWORDS = ['python', 'парсинг'] 
PAGES = 10

In [184]:
# посты с 10 первых страниц 
def get_posts(URL, pages):
    blocks = []      
    
    for page in range(1,pages+1):
        url = URL+'page'+str(page)
        req = requests.get(url)
        time.sleep(0.3)
        
        soup = BeautifulSoup(req.text, 'html.parser')
        articles  = soup.find_all('article', class_='post')
        
        blocks.append(articles)

    return blocks

post_blocks = get_posts(URL_habr, PAGES)

http://habr.com/ru/all/page1
http://habr.com/ru/all/page2
http://habr.com/ru/all/page3
http://habr.com/ru/all/page4
http://habr.com/ru/all/page5
http://habr.com/ru/all/page6
http://habr.com/ru/all/page7
http://habr.com/ru/all/page8
http://habr.com/ru/all/page9
http://habr.com/ru/all/page10
http://habr.com/ru/all/page11
http://habr.com/ru/all/page12
http://habr.com/ru/all/page13
http://habr.com/ru/all/page14
http://habr.com/ru/all/page15
http://habr.com/ru/all/page16
http://habr.com/ru/all/page17
http://habr.com/ru/all/page18
http://habr.com/ru/all/page19
http://habr.com/ru/all/page20


In [185]:
# преобразуем информацию из постов в список словарей 

def create_dictionary(blocks, pages):
    
    posts_total = []
    posts_per_page = []
    
    # каждая страница формирует свой список
    for page in range(len(post_blocks)-1):
        
        # один пост это один словарь, все посты с одной страницы объединены в список 
        for body in post_blocks[page]:
            post_dict = {}      
        
            post_dict['link'] = body.find('a', class_='post__title_link').get('href')
            post_dict['title'] =  body.find('h2', class_='post__title').text
        
            # сразу сделаем весь текст с маленькой буквы
            post_dict['text'] = body.find('div', class_='post__body post__body_crop ').text.lower()
        
            post_dict['date'] = body.find('span', class_='post__time').text
        
            posts_per_page.append(post_dict) 
            
        posts_total.append(posts_per_page) 
        
    return posts_total

post_dict = create_dictionary(post_blocks, PAGES) 

In [175]:
post_dict[0][0]

{'link': 'https://habr.com/ru/company/digital-ecosystems/blog/514860/',
 'title': '\nДайджест IT-событий первой половины сентября\n',
 'text': '\n\n\n\r\nпора отпусков подходит к концу, и мы возвращаемся к своим регулярным обзорам мероприятий, которые помогают программистам развиваться и в нерабочее время. первый учебный месяц начинается знаково – обилием обучающих курсов по разработке и другим дисциплинам. специалисты по кибербезопаности также активно обсуждают старые и новые проблемы, а крупные конференции мало-помалу находят дорогу обратно в оффлайн.\n\nчитать дальше →\n',
 'date': 'сегодня в 14:13'}

In [176]:
# получили ссылки например для всех постов, которые подгрузились
for i in range(len(post_dict)):
    for j in range(len(post_dict[i])):
        print(post_dict[i][j]['link'] )

https://habr.com/ru/company/digital-ecosystems/blog/514860/
https://habr.com/ru/post/514800/
https://habr.com/ru/post/514868/
https://habr.com/ru/post/514874/
https://habr.com/ru/post/514872/
https://habr.com/ru/company/eaton/blog/514856/
https://habr.com/ru/post/514862/
https://habr.com/ru/post/513494/
https://habr.com/ru/post/514858/
https://habr.com/ru/post/514854/
https://habr.com/ru/post/514852/
https://habr.com/ru/company/otus/blog/514848/
https://habr.com/ru/company/etmc_exponenta/blog/514846/
https://habr.com/ru/post/514844/
https://habr.com/ru/post/514826/
https://habr.com/ru/company/vdsina/blog/514692/
https://habr.com/ru/company/click/blog/514820/
https://habr.com/ru/company/oleg-bunin/blog/514834/
https://habr.com/ru/post/514840/
https://habr.com/ru/post/514838/
https://habr.com/ru/company/cloud4y/blog/514738/
https://habr.com/ru/post/514468/
https://habr.com/ru/company/microsoft/blog/514750/
https://habr.com/ru/company/vdsina/blog/514524/
https://habr.com/ru/company/vivald

In [186]:
# фильтрация
# если искомое слово есть в текст превью, то запишем данные поста в датафрейм

def condition(post_dictionary):
    habr_posts = pd.DataFrame()
    
    for i in range(len(post_dict)):
        for j in range(len(post_dict[i])):
            
            post_info = post_dictionary[i][j]
            
            if any([word in post_info['text'] for word in KEYWORDS]): 
                row = {'date': post_info['date'], 
                   'title': post_info['title'], 
                   'link': post_info['link']}
                
                habr_posts = pd.concat([habr_posts, pd.DataFrame([row])])   
            
    return habr_posts

In [187]:
condition(post_dict)

,date,link,title
0,сегодня в 10:58,https://habr.com/ru/company/oleg-bunin/blog/51...,\n«Я что-то накодил и все упало»: провалы в Py...
0,10 августа 2020 в 16:16,https://habr.com/ru/company/ruvds/blog/514538/,"\nА вы знаете о том, что в Python есть встроен..."
0,9 августа 2020 в 21:40,https://habr.com/ru/post/514444/,"\nТри редко используемых возможности Python 3,..."
0,9 августа 2020 в 08:21,https://habr.com/ru/post/514384/,\nМониторинг демон на Asyncio + Dependency Inj...
0,8 августа 2020 в 15:02,https://habr.com/ru/post/514348/,"\nШесть Python-пакетов, которые рекомендуется ..."
0,8 августа 2020 в 14:24,https://habr.com/ru/post/514342/,\nРеализация offline режима для Yandex.Music\n
0,7 августа 2020 в 13:53,https://habr.com/ru/post/514220/,\nРецепты uWSGI: преобразование документов с и...
0,7 августа 2020 в 11:27,https://habr.com/ru/post/513172/,\nTensorFlow.js: Часть 1: Использование Low-Le...
0,6 августа 2020 в 18:08,https://habr.com/ru/company/otus/blog/514132/,\nКак работают профайлеры в Ruby и Python?\n
0,6 августа 2020 в 15:19,https://habr.com/ru/post/514100/,\nПростой P2P сервер на python\n


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

##  Задание 2.

### Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:

In [18]:
import json

In [37]:
# вытащим дату и название статьи

# запишем данные в датафрейм
habr_posts = pd.DataFrame()

for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')   
    date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
    title = soup.find('h1', class_='post__title post__title_full').text
    text = post_blocks.find('div', class_='post__text post__text-html  post__text_v1').text
    
    row = {'date': date, 'title': title, 'link': link}
    
    habr_posts = pd.concat([habr_posts, pd.DataFrame([row])]) 
    
    
#text = post_blocks.find('div', class_='post__text post__text-html  post__text_v1').text
#text
post_blocks

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/Blyabtroi/" title="Автор публикации">
 <span class="default-image_mini default-image_green">
 <svg aria-hidden="true" class="icon-svg" height="24" role="img" version="1.1" viewbox="0 0 24 24" width="24"><path d="M21.5 24h-19c-1.379 0-2.5-1.122-2.5-2.5v-19c0-1.379 1.122-2.5 2.5-2.5h19c1.379 0 2.5 1.122 2.5 2.5v19c0 1.379-1.122 2.5-2.5 2.5zm-19-23c-.827 0-1.5.673-1.5 1.5v19c0 .827.673 1.5 1.5 1.5h19c.827 0 1.5-.673 1.5-1.5v-19c0-.827-.673-1.5-1.5-1.5h-19zM15.598 12.385zM19.438 15.417l-.002-.005v-.001c-.875-2.226-2.484-3.054-3.445-3.549l-.273-.143c.029-.497-.025-1.034-.167-1.599l-.128.032.123-.044c-.765-2.152-1.757-2.585-2.632-2.967l-.006-.003-.535-2.121c.357-.065.628-.375.628-.752.001-.423-.342-.765-.765-.765s-.766.342-.766.765c0 .358.248.657.581.74l-.825 1.654-.014-.003-.024-.003c-1.053-.033-1.842.369-2.5.947-.633-.322-1.515-.729-2.158-1.814.1

In [17]:
emails = ['tischa.tascha@gmail.com', 'marina88@gmail.com']
leak_url = 'https://digibody.avast.com/v1/web/leaks' 

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [23]:
json_response = []
email_leaks = pd.DataFrame()

for i in range(len(emails)):
    
    # итерация по имейлам
    payload = {'email': emails[i]}  
    
    # привести инпут к json формату 
    r = requests.post(leak_url, json=payload)
    json_response = r.json()
    
    # берем только нужную нам инфу
    leaks = json_response['value']
        
    for leak in leaks:    
        leak_date =  datetime.datetime.fromtimestamp(int(str(leak['leak_date'])[:10])).strftime('%Y-%m-%d %H:%M:%S')  
        row = {'email': payload.values(), 'date': leak_date, 'source': leak['leak_info']['title'], 'description': leak['leak_info']['description']}    
        email_leaks = pd.concat([email_leaks, pd.DataFrame([row])]) 
        
print(email_leaks)

                  date                                        description  \
0  2019-02-06 01:00:00  On January 7, 2019, an online user named Sanix...   
0  2020-06-25 02:00:00  At an unconfirmed date, this "Cash Cloud" Comb...   
0  2019-06-13 02:00:00  In May 2019, graphic-design site Canva's datab...   
0  2018-08-24 02:00:00  In 2018, education tech company Chegg's databa...   
0  2019-01-29 01:00:00  On January 7, 2019, an online user named Sanix...   
0  2019-05-16 02:00:00  In June 2018, fast-fashion retailer Shein's da...   
0  2019-01-25 01:00:00  On January 7, 2019, an online user named Sanix...   
0  2017-12-22 01:00:00  The proliferation of stolen or leaked database...   
0  2018-08-24 02:00:00  This combolist was compiled from a variety of ...   
0  2019-02-06 01:00:00  On January 7, 2019, an online user named Sanix...   
0  2020-06-25 02:00:00  At an unconfirmed date, this "Cash Cloud" Comb...   
0  2017-10-09 02:00:00  The proliferation of stolen or leaked database...   

### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.